In [1]:

import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
# Set the parent directory as the current directory
os.chdir(parent_dir)

In [5]:
import json
from collections import Counter
from typing import Dict, List, Optional, Any

def analyze_false_negatives(file_path: str, top_n: int = 10) -> List[Dict[str, Any]]:
    """
    Analyze evaluation.json file to find the most common false negatives.
    
    Args:
        file_path (str): Path to the evaluation.json file
        top_n (int): Number of top false negatives to return (default: 10)
        
    Returns:
        List[Dict[str, Any]]: List of dictionaries containing the most common false negatives,
                             with 'code' and 'count' keys
    """
    try:
        # Load the evaluation file
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Extract corpus_false_negatives based on file structure
        false_negatives = []
        
        # Check for different possible file structures
        if "exact_match" in data:
            # Newer format with exact_match and potentially fuzzy_match
            if "corpus_false_negatives" in data["exact_match"]:
                false_negatives = data["exact_match"]["corpus_false_negatives"]
        elif "corpus_false_negatives" in data:
            # Older format with direct corpus_false_negatives
            false_negatives = data["corpus_false_negatives"]
        elif "results" in data and "corpus_false_negatives" in data["results"]:
            # Nested results format
            false_negatives = data["results"]["corpus_false_negatives"]
        else:
            # Try to find error_analysis section
            if "error_analysis" in data and "most_common_false_negatives" in data["error_analysis"]:
                return data["error_analysis"]["most_common_false_negatives"][:top_n]
            elif "exact_match" in data and "error_analysis" in data["exact_match"]:
                if "most_common_false_negatives" in data["exact_match"]["error_analysis"]:
                    return data["exact_match"]["error_analysis"]["most_common_false_negatives"][:top_n]
            else:
                raise ValueError("Could not find false negatives in the evaluation file")
        
        # Count occurrences of each false negative
        fn_counter = Counter()
        
        for fn in false_negatives:
            code = fn.get("code", "")
            count = fn.get("count", 1)  # Default to 1 if count not specified
            fn_counter[code] += count
        
        # Get most common false negatives
        most_common = [
            {"code": code, "count": count}
            for code, count in fn_counter.most_common(top_n)
        ]
        
        return most_common
        
    except Exception as e:
        print(f"Error analyzing false negatives: {e}")
        return []

def print_false_negatives(file_path: str, top_n: int = 10) -> None:
    """
    Print the most common false negatives from an evaluation.json file.
    
    Args:
        file_path (str): Path to the evaluation.json file
        top_n (int): Number of top false negatives to print (default: 10)
    """
    most_common = analyze_false_negatives(file_path, top_n)
    
    if not most_common:
        print("No false negatives found or could not analyze the file.")
        return
    
    print("\nMost Common False Negative ORPHA Codes:")
    print("-" * 40)
    for i, item in enumerate(most_common, 1):
        print(f"{i}. {item['code']} ({item['count']} occurrences)")
    print()

# Example usage in a notebook:
file_path = 'data/results/agents/rd/evaluation/multistage_min_500_correction.json'
print_false_negatives(file_path)

Error analyzing false negatives: Could not find false negatives in the evaluation file
No false negatives found or could not analyze the file.
